# Análise do Dataset

1. Quais são os 15 produtos mais vendidos em termos de quantidade e receita?

2. Quais são os 15 produtos mais cancelados no período em termos de quantidade e receita?

3. Quais são os 15 produtos com mais ocorrências de cancelamento no período?

4. Qual são os paises com o maiores volumes de vendas? E o paises com o maiores valores médios de compra por transação?

5. Quais são os padrões sazonais nas vendas ao longo do ano?

6. Como as vendas variam ao longo da semana ou do mês?

7. Quais são os produtos mais frequentemente comprados em conjunto?

8. Existe uma correlação entre o tamanho do pedido (quantidade de itens) e o valor total da compra?

9. Quais são os clientes mais frequentes e quem são os maiores gastadores?

10. Qual é o valor médio de compra por cliente?

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# Carregando e lendo o dataset
sales = pd.read_csv('C:/Users/mathe/Projetos Dados/EcommerceSalesAnalysis/App/Data/Online Retail.csv', delimiter=';', encoding='latin1', decimal=',')
sales.head()

In [ ]:
sales.shape

Uma análise do Dataset mostrou que além de registrar as operações de venda e cancelamento dos produtos, esse dataset também registra operações manuais de ajustes, descontos ofertados, taxas, comissões e outras operações.

Portanto, como a análise está focada apenas na VENDA e CANCELAMENTO dos PRODUTOS, os registros citados serão excluídos.

Essas operações são identificadas pela coluna *[StockCode]*, em que seus valores iniciam-se por uma letra.

In [ ]:
# Filtrando as operações que não se referem a produtos
def not_number(value):
    return not value[0].isdigit()

# Filtra todos os registros que não vendas/cancelamento de produtos
not_product_lines = sales[sales['StockCode'].apply(not_number)]

# Obter os valores únicos das linhas
not_product = not_product_lines['StockCode'].unique()

# Imprimir os valores únicos que não começam com números
print(not_product)

In [ ]:
# Filtrando o dataset apenas por venda e cancelamento de produtos
sales = sales[~sales['StockCode'].isin(not_product)]

In [ ]:
# Conhecendo os tipos dos dados
sales.dtypes

In [ ]:
# Adicionando coluna de Preço Final de venda/devolução/descontos ou taxas para cada transação
sales['FinalPrice'] = sales['Quantity'] * sales['UnitPrice']

In [ ]:
# Resumo dos dados
sales[['Quantity', 'UnitPrice', 'FinalPrice']].describe().T

In [ ]:
# Excluindo os registros com valor unitário igual a zero
sales = sales[sales['UnitPrice'] > 0]
sales.shape

In [ ]:
# Verifica a presença de registros duplicados no dataset
cols_list = sales.columns
duplicates = sales.duplicated(subset=cols_list, keep=False)
sales[duplicates].sort_values('Description').head()

In [ ]:
# Remove as linhas duplicadas e retorna o novo tamanho do dataset
sales_processed = sales.drop_duplicates()
sales_processed.shape

In [ ]:
# Checando valores ausentes
sales_processed.isna().sum()

Há presença de valores ausentes na coluna *[CustomerID].*

A coluna de *[CustomerID]* só será utilizada nas duas últimas análises, logo, as linhas com valores ausentes serão mantidas na análise inicial.

In [ ]:
# Verificando a existência de nomes de produtos iguais para diferentes StockCodes
duplicates_descriptions = sales_processed.groupby('Description')['StockCode'].nunique()

# Filtra os casos onde há mais de um StockCode para a mesma Description
duplicates_descriptions = duplicates_descriptions[duplicates_descriptions > 1]

# Imprime as Description com mais de um StockCode associado
print("Descriptions com mais de um StockCode associado:")
print(duplicates_descriptions.sort_values(ascending=False))

In [ ]:
# Verificando a existência de StockCodes iguais para diferentes Description
duplicates_stock_codes = sales_processed.groupby('StockCode')['Description'].nunique()

# Filtra os casos onde há mais de um Description para um mesmo StockCode
duplicates_stock_codes = duplicates_stock_codes[duplicates_stock_codes > 1]

# Imprime os StockCodes com mais de um Description associado
print("StockCodes com mais de um Description associado:")
print(duplicates_stock_codes.sort_values(ascending=False))

Há dois casos observados:
1. Um mesmo código possui descrições diferentes
2. Códigos diferentes possuem descrições iguais.

Para a análise feita neste notebook, será considerado que esses casos são decorrentes de cadastros errados, duplicados, registros incorretos ou alterações propositais nas descrições. 

In [ ]:
# Quantidade de valores únicos da coluna Description
sales_processed.nunique()

Para simplificar a análise e melhorar a identificação dos produtos, cada registro único de *[Description]* será considerado como um produto diferente, já que a quantidade valores únicos de StockCode e Description são bem próximos.

Observa-se, portanto, um total de 4019 tipos de produtos distintos

In [ ]:
sales_processed.to_csv('C:/Users/mathe/Projetos Dados/EcommerceSalesAnalysis/App/Data/OnlineRetail_processed.csv', index=False, encoding='utf-8')